# Review

Congratulations! You’ve now learned how to use simulation to investigate the trade-offs for an A/B test sample-size calculation. As a recap, this lesson covered the following:

- The significance threshold for a test is equal to the false positive rate
- The power of a test is the probability of correctly detecting a significant result
- Increasing sample size increases the power of a test
- Increasing the significance threshold increases power, but also increases the false positive rate
- Larger sample sizes are needed to detect smaller effect sizes

Two notes about the terminology in the sample size calculator:

1. **Baseline conversion rate** is equivalent to our `control_rate` in the code.
2. **Minimum detectable effect (MDE)** is the smallest effect size (or `lift`) that we want our test to be able to detect. If the MDE is larger than our true `lift`, power will decrease because our sample size might not be large enough to detect the difference between the two groups.



## Instructions

1. As a final exercise, we’ve provided a sample size calculator for an A/B test, along with the simulation code from the previous exercises. The calculator estimates the sample size needed to achieve 80% power. Plug in the following values to the sample size calculator:

    - Baseline rate: 50%
    - Minimum detectable effect: 30%
    - Significance threshold: 5%

    Then, set the sample size for the simulation code equal to the sample size indicated by the calculator. Press “Run” and inspect the proportion of tests that were significant. The proportion should be close to 0.80!

    <details>
        <summary>Stuck? Get a hint</summary>
    
    Make sure to change all three criteria in the calculator, and update the `sample_size` variable in the code to match the output of the calculator.
    </details>

2. Let’s now examine how MDE impacts the power of our test. Change the MDE in the calculator to 40% so that you have:

    - Baseline rate: 50%
    - Minimum detectable effect: 40%
    - Significance threshold: 5%

    Update the `sample_size` in our simulator to match the new sample size given by the calculator. Press “Run” and inspect the proportion of tests that were significant. Now that our MDE is larger than our actual effect, what do you see happens to our power?

    <details>
        <summary>Stuck? Get a hint</summary>
    
    When the Minimum Detectable Effect is larger than our actual effect, power decreases as our test does not have a large enough sample size to detect the small effect (AKA lift).
    </details>


In [1]:
from html import escape
 
class myDisplayObject:
    def __init__(self, html, width="100%", height="200px", ratio=1):
        self.width = width
        self.height = height
        self.ratio = ratio
        self.html = html
 
    # cribbed from branca Py package
    def _repr_html_(self, **kwargs):
        """Displays the Diagram in a Jupyter notebook."""
        html = escape(self.html)
        iframe = (
            '<iframe srcdoc="{html}" width="{width}" height="{height}"'
            'style="border:none !important;" '
            '"allowfullscreen" "webkitallowfullscreen" "mozallowfullscreen">'
            '</iframe>'
        ).format(html=html, width=self.width, height=self.height)
        return iframe

myDisplayObject(open('../../assets/ab_ss_calculator.html', 'r').read(), height="220px")

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

# preset values
significance_threshold = 0.05
sample_size = 100#change the sample size here
lift = .3
control_rate = .5
name_rate = (1 + lift) * control_rate

# initialize an empty list of results
results = []

# start the loop
for i in range(100):
  # simulate data:
  sample_control = np.random.choice(['yes', 'no'],  size=int(sample_size/2), p=[control_rate, 1-control_rate])
  sample_name = np.random.choice(['yes', 'no'], size=int(sample_size/2), p=[name_rate, 1-name_rate])
  group = ['control']*int(sample_size/2) + ['name']*int(sample_size/2)
  outcome = list(sample_control) + list(sample_name)
  sim_data = {"Email": group, "Opened": outcome}
  sim_data = pd.DataFrame(sim_data)

  # run the test
  ab_contingency = pd.crosstab(np.array(sim_data.Email), np.array(sim_data.Opened))
  chi2, pval, dof, expected = chi2_contingency(ab_contingency)
  result = ('significant' if pval < significance_threshold else 'not significant')

  # append the result to our results list:
  results.append(result)

# calculate proportion of significant results:
print("Proportion of significant results:")
results =  np.array(results)
print(np.sum(results == 'significant')/100)